# Importations

In [9]:
from __future__ import print_function

# Standard libraries
import os
import time
import random
import warnings
from math import sqrt, pow

# Data processing
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Machine learning
import sklearn
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GroupKFold, GroupShuffleSplit
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.decomposition import PCA

# Deep learning
import tensorflow as tf
from tensorflow.keras import Sequential, optimizers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Dropout, LSTM, Activation, GRU
from tensorflow.keras.callbacks import EarlyStopping
import keras
from keras import backend as K

# Optimization
from scipy import optimize
from scipy.stats import spearmanr
from scipy.spatial.distance import pdist, squareform
from hyperopt import fmin, tpe, Trials, hp, STATUS_OK, space_eval


%matplotlib inline
warnings.filterwarnings('ignore')

SEED = 0
def set_seed(seed=SEED):
    os.environ['PYTHONHASHSEED'] = str(SEED)
    random.seed(SEED)
    np.random.seed(SEED)
    tf.random.set_seed(SEED)

# Appeler la fonction pour fixer le seed
set_seed(SEED)


# Methode.py

In [10]:
# read the train and test data
def prep_data(train, test, drop_sensors, remaining_sensors, alpha, drop = True):
    if drop:
        X_train_interim = add_operating_condition(train.drop(drop_sensors, axis=1))
        X_test_interim = add_operating_condition(test.drop(drop_sensors, axis=1))
    else:
        X_train_interim = add_operating_condition(train)
        X_test_interim = add_operating_condition(test)

    X_train_interim, X_test_interim = condition_scaler(X_train_interim, X_test_interim, remaining_sensors)
    X_train_interim = exponential_smoothing(X_train_interim, remaining_sensors, 0, alpha)
    X_test_interim = exponential_smoothing(X_test_interim, remaining_sensors, 0, alpha)

    return X_train_interim, X_test_interim

def rul_piecewise_fct(X_train, rul):

    X_train['RUL'].clip(upper=rul, inplace=True)

    return X_train

def prepare_data(file_name):
    dir_path = '../data/'
    dependent_var = ['RUL']
    index_names = ['Unit', 'Cycle']
    setting_names = ['Altitude', 'Mach', 'TRA']
    sensor_names = ['T20','T24','T30','T50','P20','P15','P30','Nf','Nc','epr','Ps30','phi',
                    'NRf','NRc','BPR','farB','htBleed','Nf_dmd','PCNfR_dmd','W31','W32']
    col_names = index_names + setting_names + sensor_names

    df_train = pd.read_csv(dir_path+'train_'+str(file_name),delim_whitespace=True,names=col_names)

    rul_train = pd.DataFrame(df_train.groupby('Unit')['Cycle'].max()).reset_index()
    rul_train.columns = ['Unit', 'max']
    df_train = df_train.merge(rul_train, on=['Unit'], how='left')
    df_train['RUL'] = df_train['max'] - df_train['Cycle']
    df_train.drop('max', axis=1, inplace=True)

    df_test = pd.read_csv(dir_path+'test_'+str(file_name), delim_whitespace=True, names=col_names)

    y_test = pd.read_csv(dir_path+'RUL_'+(file_name), delim_whitespace=True,names=["RUL"])
    #y_true["Unit"] = y_true.index + 1
    return df_train, df_test, y_test


# add operational condition to then normalize the data based on these operational conditions test
def add_operating_condition(df):
    df_op_cond = df.copy()

    df_op_cond['Altitude'] = df_op_cond['Altitude'].round()
    df_op_cond['Mach'] = df_op_cond['Mach'].round(decimals=2)
    df_op_cond['TRA'] = df_op_cond['TRA'].round()

    # converting settings to string and concatanating makes the operating condition into a categorical variable
    df_op_cond['op_cond'] = df_op_cond['Altitude'].astype(str) + '_' + \
                        df_op_cond['Mach'].astype(str) + '_' + \
                        df_op_cond['TRA'].astype(str)

    return df_op_cond

# normalize the data based on the operational condition
def condition_scaler(df_train, df_test, sensor_names):
  # apply operating condition specific scaling
  #scaler = StandardScaler()
    scaler = MinMaxScaler(feature_range = (0, 1))
    for condition in df_train['op_cond'].unique():
        scaler.fit(df_train.loc[df_train['op_cond']==condition, sensor_names])
        df_train.loc[df_train['op_cond']==condition, sensor_names] = scaler.transform(df_train.loc[df_train['op_cond']==condition, sensor_names])
        df_test.loc[df_test['op_cond']==condition, sensor_names] = scaler.transform(df_test.loc[df_test['op_cond']==condition, sensor_names])
    return df_train, df_test




# denoise the signal using the exponential signal wih an alpha equals to 0.3
def exponential_smoothing(df, sensors, n_samples, alpha=0.2):
    df = df.copy()
    # first, calculate the exponential weighted mean of desired sensors
    new_column = df.groupby('Unit')[sensors].apply(lambda x: x.ewm(alpha=alpha).mean())
    df[sensors] = new_column.reset_index(level=0, drop=True)


    # second, drop first n_samples of each unit_nr to reduce filter delay
    def create_mask(data, samples):
        result = np.ones_like(data)
        result[0:samples] = 0
        return result

    mask = df.groupby('Unit')['Unit'].transform(create_mask, samples=n_samples).astype(bool)
    df = df[mask]

    return df

def root_mean_squared_error(y_true, y_pred):
    return np.sqrt(np.mean(np.square(y_pred - y_true)))

#the score defined in the paper
def compute_s_score(rul_true, rul_pred):
    diff = rul_pred - rul_true
    return np.sum(np.where(diff < 0, np.exp(-diff/13)-1, np.exp(diff/10)-1))

#evaluate the model with R² and RMSE
def evaluate(y_true, y_hat, label='test'):
    mse = mean_squared_error(y_true, y_hat)
    rmse = np.sqrt(mse)
    variance = r2_score(y_true, y_hat)
    print('{} set RMSE:{}, R2:{}'.format(label, rmse, variance))

def generate_sequences(data, sequence_length):
    """
    Generate sequences of a given length from the input data.
    """
    num_samples = data.shape[0]

    # Generate sequences using sliding windows
    for start_idx in range(num_samples - sequence_length + 1):
        end_idx = start_idx + sequence_length
        yield data[start_idx:end_idx, :]

def generate_data_wrapper(df, sequence_length, columns, unit_nrs=None):
    """
    Wrapper function to generate sequences for multiple units in the dataset.
    """
    if unit_nrs is None:
        unit_nrs = df['Unit'].unique()

    # Generate sequences for each unit and concatenate them
    all_sequences = []
    for unit_nr in unit_nrs:
        unit_data = df[df['Unit'] == unit_nr][columns].values
        sequences = list(generate_sequences(unit_data, sequence_length))
        all_sequences.extend(sequences)

    return np.array(all_sequences, dtype=np.float32)


def gen_train_data(df, sequence_length, columns):
    data = df[columns].values
    num_elements = data.shape[0]

    # -1 and +1 because of Python indexing
    for start, stop in zip(range(0, num_elements-(sequence_length-1)), range(sequence_length, num_elements+1)):
        yield data[start:stop, :]

def gen_data_wrapper(df, sequence_length, columns, unit_nrs=np.array([])):
    if unit_nrs.size <= 0:
        unit_nrs = df['Unit'].unique()

    data_gen = (list(gen_train_data(df[df['Unit']==unit_nr], sequence_length, columns))
               for unit_nr in unit_nrs)
    data_array = np.concatenate(list(data_gen)).astype(np.float32)
    return data_array

def create_model(TW , remaining_):
#     history = History()
    model = Sequential()
    model.add(LSTM(units=128, activation='tanh',input_shape=(TW, len(remaining_))))
    model.add(Dense(units=128, activation='relu'))
    #model.add(GlobalAveragePooling1D(name = 'feature_layer'))
    model.add(Dropout(0.1))
    model.add(Dense(1, activation='relu'))
    model.compile(loss='mse',metrics=['mse'], optimizer=tf.keras.optimizers.Adam(learning_rate=0.01))

    return model

def compute_MAPE(y_true, y_hat):
    mape = np.mean(np.abs((y_true - y_hat)/y_true))*100
    return mape

def gen_labels(df, sequence_length, label):
    data_matrix = df[label].values
    num_elements = data_matrix.shape[0]

    # -1 because I want to predict the rul of that last row in the sequence, not the next row
    return data_matrix[sequence_length-1:num_elements, :]

def gen_label_wrapper(df, sequence_length, label, unit_nrs=np.array([])):
    if unit_nrs.size <= 0:
        unit_nrs = df['Unit'].unique()

    label_gen = [gen_labels(df[df['Unit']==unit_nr], sequence_length, label)
                for unit_nr in unit_nrs]
    label_array = np.concatenate(label_gen).astype(np.float32)
    return label_array
def gen_test_data(df, sequence_length, columns, mask_value):
    if df.shape[0] < sequence_length:
        data_matrix = np.full(shape=(sequence_length, len(columns)), fill_value=mask_value) # pad
        idx = data_matrix.shape[0] - df.shape[0]
        data_matrix[idx:,:] = df[columns].values  # fill with available data
    else:
        data_matrix = df[columns].values

    # specifically yield the last possible sequence
    stop = num_elements = data_matrix.shape[0]
    start = stop - sequence_length
    for i in list(range(1)):
        yield data_matrix[start:stop, :]


def new_column (df, column):
    #df = df.sort_values(by=column, ascending=False)
    df[column] = range(1, len(df) + 1)
    return df

# Préparation des données et configuration initiale

In [ ]:
train, test, y_test = prepare_data('FD004.txt')
print(train.shape, test.shape, y_test.shape)
sensor_names = ['T20','T24','T30','T50','P20','P15','P30','Nf','Nc','epr','Ps30','phi',
                    'NRf','NRc','BPR','farB','htBleed','Nf_dmd','PCNfR_dmd','W31','W32']

remaining_sensors = ['T24','T30','T50', 'P15', 'P30','Nf','Nc', 'epr','Ps30','phi',
                     'NRf','NRc','BPR', 'farB','htBleed','W31','W32']
drop_sensors = [element for element in sensor_names if element not in remaining_sensors]

rul_piecewise = 120
train['RUL'].clip(upper=rul_piecewise, inplace=True)

# Configuration des paramètres
alpha = 0.2
sequence_length = 40
epochs = 10
#nodes_per_layer = [64]
#dropout = 0.2
activation = 'tanh'
batch_size = 32
remaining_sensors = remaining_sensors
input_shape = (sequence_length, len(remaining_sensors))

#preciser la regle utilisee avec la ref(auteur)
#hidden_size_list = [32, 64, 128, 256]
space_val = {
    'hidden_size': {
        'min': 32,
        'max': 256,
        'step': 32
    },
    'learning_rate': {
        'min': np.log(1e-5),
        'max': np.log(1e-2),
        'num': 10
    },
    'dropout': {
        'min': 0.1,
        'max': 0.5,
        'step': 0.1
    }
}

# Préparation des données
X_train_interim, X_test_interim = prep_data(train, test, drop_sensors, remaining_sensors, alpha)
train_array = generate_data_wrapper(X_train_interim, sequence_length, remaining_sensors)
label_array = gen_label_wrapper(X_train_interim, sequence_length, ['RUL'])

test_gen = (list(gen_test_data(X_test_interim[X_test_interim['Unit']==unit_nr], sequence_length, remaining_sensors, -99.))
               for unit_nr in X_test_interim['Unit'].unique())
test_array = np.concatenate(list(test_gen)).astype(np.float32)

test_rul = rul_piecewise_fct(y_test, rul_piecewise)
print(train_array.shape, label_array.shape, test_array.shape)

path_bayesian = '../data/grid_bayesian_random20/fd004_bayesian20.csv'

(61249, 27) (41214, 26) (248, 1)
(51538, 40, 17) (51538, 1) (248, 40, 17)


# Bayesian Search

## Creation et entrainement du modele

In [12]:
def model_lstm_1layer(input_shape, nodes_per_layer, dropout, activation, learning_rate):
    model = Sequential()
    model.add(LSTM(units=int(nodes_per_layer), activation=activation, input_shape=input_shape))
    model.add(Dropout(dropout))
    model.add(Dense(256))
    model.add(Dense(1))  # Sortie pour la régression
    model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=learning_rate))
    return model

# Fonction pour entraîner le modèle et évaluer la RMSE
def train_model(params):
    # Création du modèle
    model = model_lstm_1layer(input_shape, params['hidden_size'], params['dropout'], activation, params['learning_rate'])

    # Entraînement du modèle
    history = model.fit(
        train_array, label_array,
        validation_data=(test_array, test_rul),
        epochs=3,
        batch_size=32,
        verbose=0
    )

    # Prédiction sur l'ensemble de validation
    y_pred = model.predict(test_array)

    # Calcul de la RMSE, S-Score, Mape
    rmse = sqrt(mean_squared_error(test_rul, y_pred))
    s_score = compute_s_score(test_rul, y_pred)
    mape = compute_MAPE(test_rul, y_pred)
    r2 = r2_score(test_rul, y_pred)

    # Retourner la RMSE comme métrique à minimiser
    return {'loss': rmse, 'status': STATUS_OK, 's_score':s_score, 'mape':mape, 'rmse':rmse}

## Apply HyperOpt TPE and store the combination of hyperparameters

In [13]:
# Initialize results dataframe with all relevant columns
results_all = pd.DataFrame()
set_seed(42)

# Define the search space
space = {
    'hidden_size': hp.quniform('hidden_size',
                              space_val['hidden_size']['min'],
                              space_val['hidden_size']['max'],
                              space_val['hidden_size']['step']),
    'learning_rate': hp.loguniform('learning_rate',
                              space_val['learning_rate']['min'],
                              space_val['learning_rate']['max']),
    'dropout': hp.quniform('dropout',
                           space_val['dropout']['min'],
                           space_val['dropout']['max'],
                           space_val['dropout']['step'])
}

# Run Bayesian optimization multiple times
def run_optimization(num_runs=20):
    global results_all

    for run_id in range(1, num_runs + 1):
        print(f"\nStarting optimization run {run_id}/{num_runs}")
        start_time = time.time()

        trials = Trials()

        best = fmin(
            fn=train_model,
            space=space,
            algo=tpe.suggest,
            max_evals=10,  # Number of evaluations per run
            trials=trials
        )

        # Get best trial results
        decoded_best = space_eval(space, best)

        print("Best Hyperparameters:")
        print(decoded_best)

        best_trial = trials.best_trial
        time_training = time.time() - start_time

        best_rmse = trials.best_trial['result']['rmse']
        best_s_score = trials.best_trial['result']['s_score']
        best_mape = trials.best_trial['result']['mape']
        best_r2 = trials.best_trial['result']['loss']


        new_result = {
          'id': run_id,
          'hidden_size': decoded_best['hidden_size'],
          'learning_rate': best['learning_rate'],
          'dropout': best['dropout'],
          'rmse': best_rmse,
          's_score': best_s_score,
          'mape': best_mape,
          'r2': -best_r2,
          'training_time': time_training
      }
        print(results_all)

        # Update results dataframe
        results_all = pd.concat([results_all, pd.DataFrame([new_result])], ignore_index=True)

        # Save to CSV after each run
        results_all.to_csv(path_bayesian, index=False)

    return results_all

# Execute the optimization 20 times
final_results = run_optimization(num_runs=20)

print("\nFinal Results:")
print(final_results)


Starting optimization run 1/20
  0%|          | 0/10 [00:00<?, ?trial/s, best loss=?]

  0%|          | 0/10 [00:26<?, ?trial/s, best loss=?]


KeyboardInterrupt: 

## Visualisation de la moyenne et l'ecart type

# Random Search

In [ ]:
def model_lstm_1layer(input_shape, nodes_per_layer, dropout, activation, learning_rate):
    model = Sequential()
    model.add(LSTM(units=int(nodes_per_layer), activation=activation, input_shape=input_shape))
    model.add(Dropout(dropout))
    model.add(Dense(256))
    model.add(Dense(1))  # Output for regression
    model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=learning_rate))
    return model

# Number of random trials
n_trials = 20

# Container for results
results_all = pd.DataFrame()

# Random Search
for i in range(n_trials):
    # Sample hyperparameters
    #hidden_size = random.randint(space_val['hidden_size']['min'], space_val['hidden_size']['max'])
    #hidden_size =  random.choice([32, 64, 128, 256]),
    hidden_size = round(random.uniform(
        space_val['hidden_size']['min'],
        space_val['hidden_size']['max']
    ) / space_val['hidden_size']['step']) * space_val['hidden_size']['step']

    learning_rate = np.exp(random.uniform(space_val['learning_rate']['min'], space_val['learning_rate']['max']))
    dropout = round(random.uniform(
        space_val['dropout']['min'],
        space_val['dropout']['max']
    ) / space_val['dropout']['step']) * space_val['dropout']['step']

    print(f"Trial {i+1}/{n_trials} - LSTM units={hidden_size}, LR={learning_rate:.5f}, dropout={dropout}")

    start_time = time.time()

    # Build and train model
    model = model_lstm_1layer(input_shape, hidden_size, dropout, activation, learning_rate)
    history = model.fit(
        train_array, label_array,
        validation_data=(test_array, test_rul),
        epochs=epochs,
        batch_size=batch_size,
        verbose=0
    )

    # Evaluate
    y_pred = model.predict(test_array)
    rmse = root_mean_squared_error(test_rul, y_pred)
    score = compute_s_score(test_rul, y_pred)
    mape = compute_MAPE(test_rul, y_pred)
    r2 = r2_score(test_rul, y_pred)
    training_time = time.time() - start_time

    print(f"→ RMSE: {rmse:.4f}, Time: {training_time:.2f}s")

    # Save to results
    results_all = pd.concat([results_all, pd.DataFrame([{
        'id': i + 1,
        'hidden_size': hidden_size,
        'learning_rate': learning_rate,
        'dropout': dropout,
        'rmse': rmse,
        's-score': score,
        'mape': mape,
        'r2': r2,
        'training_time': training_time
    }])], ignore_index=True)

    # Save to CSV
    results_all.to_csv(path_random, index=False)


Trial 1/20 - LSTM units=224, LR=0.00801, dropout=0.2
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
→ RMSE: 19.1663, Time: 253.96s
Trial 2/20 - LSTM units=64, LR=0.00002, dropout=0.4
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
→ RMSE: 25.1781, Time: 107.02s
Trial 3/20 - LSTM units=64, LR=0.00002, dropout=0.1
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
→ RMSE: 21.2680, Time: 106.52s
Trial 4/20 - LSTM units=96, LR=0.00009, dropout=0.4
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
→ RMSE: 26.8661, Time: 150.48s
Trial 5/20 - LSTM units=96, LR=0.00260, dropout=0.30000000000000004
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
→ RMSE: 23.4732, Time: 148.39s
Trial 6/20 - LSTM units=224, LR=0.00101, dropout=0.1
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
→ RMSE: 13.2279, Time: 275.48s
Trial 7/20 - LSTM units=96, LR=0.00409, dropout=0.2
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
→ RMSE: 13.2683, Time: 145.23s
Trial 8/20 - LSTM units=160, LR=0.00002, dropout=0.5
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
→ RMSE: 29.3626, Time: 201.41s
Trial 9/20 - 

# Grid Search

In [ ]:
# Generate grid values from space_val
param_grid = {
    'hidden_size': [32, 64, 96, 128, 160, 192, 224, 256],

    'learning_rate': [np.exp(x) for x in np.linspace(
        space_val['learning_rate']['min'],
        space_val['learning_rate']['max'],
        num=space_val['learning_rate']['num']
    )],
    'dropout': np.round(np.arange(
        space_val['dropout']['min'],
        space_val['dropout']['max'] + 0.001,
        space_val['dropout']['step']
    ), 2)
}
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)
# -------------------------
# LSTM Model Function
# -------------------------
def model_lstm_1layer(input_shape, nodes_per_layer, dropout, activation, learning_rate):
    model = Sequential()
    model.add(LSTM(units=int(nodes_per_layer), activation=activation, input_shape=input_shape))
    model.add(Dropout(dropout))
    model.add(Dense(256))
    model.add(Dense(1))  # Sortie pour la régression
    model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=learning_rate))
    return model

# -------------------------
# Grid Search
# -------------------------
results_all = pd.DataFrame()
i = 0
set_seed(SEED)
for hidden_size in param_grid['hidden_size']:
    for learning_rate in param_grid['learning_rate']:
        for dropout in param_grid['dropout']:
            print(f"Training with LSTM units={hidden_size}, learning_rate={learning_rate:.4f}, dropout={dropout}")

            with tf.device('/device:GPU:0'):
                start_time = time.time()
                # Build and train model
                model = model_lstm_1layer(input_shape, hidden_size, dropout, activation, learning_rate)
                history = model.fit(
                    train_array, label_array,
                    validation_data=(test_array, test_rul),
                    epochs=epochs,
                    batch_size=batch_size,
                    verbose=0,
                    callbacks=[early_stop]
                )

                # Predict and evaluate
                y_pred = model.predict(test_array)
                rmse = root_mean_squared_error(test_rul, y_pred)
                s_score = compute_s_score(test_rul, y_pred)
                mape = compute_MAPE(test_rul, y_pred)
                r2 = r2_score(test_rul, y_pred)

            print(f"Validation RMSE: {rmse:.4f}")
            time_training = time.time() - start_time
            i += 1

            # Save results
            results_all = pd.concat([results_all, pd.DataFrame([{
                'id': i,
                'hidden_size': hidden_size,
                'learning_rate': learning_rate,
                'dropout': dropout,
                'rmse': rmse,
                's_score': s_score,
                'mape': mape,
                'r2': r2,
                'training_time': time_training
            }])], ignore_index=True)

            results_all.to_csv(path_grid, index=False)

Training with LSTM units=32, learning_rate=0.0000, dropout=0.1
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Validation RMSE: 27.8205
Training with LSTM units=32, learning_rate=0.0000, dropout=0.2
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Validation RMSE: 24.3584
Training with LSTM units=32, learning_rate=0.0000, dropout=0.3
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Validation RMSE: 30.1438
Training with LSTM units=32, learning_rate=0.0000, dropout=0.4
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Validation RMSE: 24.8540
Training with LSTM units=32, learning_rate=0.0000, dropout=0.5
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Validation RMSE: 27.0758
Training with LSTM units=32, learning_rate=0.0000, dropout=0.1
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Validation RMSE: 22.5673
Training with LSTM units=32, learning_rate=0.0000, dropout=0.2
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Validation RMSE: 19.8573
Training with LSTM units=32, learning_rate=0.0000, dropout=0.3
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Validation RMSE: 

# v

In [ ]:
df = pd.read_csv(path_bayesian)
plt.figure(figsize=(10, 5))
plt.plot(
    df["id"],
    df["rmse"],
    marker="o",
    linestyle="-",
    color="blue",
    linewidth=2,
    markersize=8,
    label="RMSE"
)

# Highlight the best RMSE (lowest value)
best_idx = df["rmse"].idxmin()
plt.scatter(
    df.loc[best_idx, "id"],
    df.loc[best_idx, "rmse"],
    color="red",
    s=100,
    label=f"Best RMSE: {df.loc[best_idx, 'rmse']:.2f}",
    zorder=5
)

# Customize the plot
plt.title("RMSE Across Hyperparameter Configurations", fontsize=14)
plt.xlabel("id", fontsize=12)
plt.ylabel("RMSE", fontsize=12)
plt.xticks(df["id"])
plt.grid(True, linestyle="--", alpha=0.7)
plt.legend(fontsize=10)
plt.tight_layout()
plt.show()

In [ ]:
df = pd.read_csv(path_random)
plt.figure(figsize=(10, 5))
plt.plot(
    df["id"],
    df["rmse"],
    marker="o",
    linestyle="-",
    color="blue",
    linewidth=2,
    markersize=8,
    label="RMSE"
)

# Highlight the best RMSE (lowest value)
best_idx = df["rmse"].idxmin()
plt.scatter(
    df.loc[best_idx, "id"],
    df.loc[best_idx, "rmse"],
    color="red",
    s=100,
    label=f"Best RMSE: {df.loc[best_idx, 'rmse']:.2f}",
    zorder=5
)

# Customize the plot
plt.title("RMSE Across Hyperparameter Configurations", fontsize=14)
plt.xlabel("ID", fontsize=12)
plt.ylabel("RMSE", fontsize=12)
plt.xticks(df["id"])
plt.grid(True, linestyle="--", alpha=0.7)
plt.legend(fontsize=10)
plt.tight_layout()
plt.show()